In [ ]:
from __future__ import annotations
import datetime as dt
from typing import NamedTuple
from collections.abc import Iterable
from random import random
import sys

from micro_namedtuple_sqlite_persister.persister import Engine
from micro_namedtuple_sqlite_persister.adaptconvert import enable_included_adaptconverters
enable_included_adaptconverters()

# Models

In [ ]:
class MyModel(NamedTuple):
    id: int | None
    name: str
    date: dt.datetime
    score: float | None

Connect to the database and create tables with an `Engine`

In [ ]:
engine = Engine(":memory:")
# engine = Engine("example.db")
engine.ensure_table_created(MyModel, force_recreate=True)
engine.connection.set_trace_callback(lambda sql: print(sql, file=sys.stderr)) # echo SQL
engine.connection # just the real connection object

# Basic CRUD

## Insert row

In [ ]:
row = MyModel(None, "Bart", dt.datetime.now(), 6.5)
row = engine.insert(row)
engine.connection.commit()
row

## Get row by id

In [ ]:
engine.get(MyModel,row.id)

## Update row

In [ ]:
engine.update(row._replace(score=78.9))
engine.connection.commit()

## Delete row

by id

In [ ]:
row2 = engine.insert(MyModel(None, "foo", dt.datetime.now(), 6.5))

engine.delete(MyModel, row2.id)
engine.connection.commit()

by instance

In [ ]:
row3 = engine.insert(MyModel(None, "bar", dt.datetime.now(), 9.5))

engine.delete(row3)
engine.connection.commit()

# Foreign Keys Relationships
Models can be related by using a model as a field type in another model.

In [ ]:
class Band(NamedTuple):
    id: int | None
    name: str
    active: bool

class BandMember(NamedTuple):
    id: int | None
    band: Band
    name: str
    instrument: str

engine.ensure_table_created(Band)
engine.ensure_table_created(BandMember)

## Save
Recursively insert/update/get related models. Model without an id will be inserted, model with an id will be updated.

In [ ]:
beatles = Band(None, "The Band", True)
paul = BandMember(None, beatles, "Paul McCartney", "Bass")

# saves both the band and the member
paul = engine.insert(paul)
engine.connection.commit()

## Recursive loading
Related models are loaded recursively.

You can control/disable this by making view models that exclude the related models.

In [ ]:
singer = engine.get(BandMember, paul.id)
display(singer)

# Note how the `band` field gets pulled in
print(singer.name)
print(singer.band.name)

In [ ]:
class League(NamedTuple):
    id: int | None
    leaguename: str

class Team(NamedTuple):
    id: int | None
    teamname: str
    league: League

class Athlete(NamedTuple):
    id: int | None
    name: str
    team: Team


engine.ensure_table_created(League)
engine.ensure_table_created(Team)
engine.ensure_table_created(Athlete)

# Insert dummy data
leagues = [
    engine.save(League(None, "Big")),
    engine.save(League(None, "Small")),
    ]
teams = [
    red:=engine.save(Team(None, "Red", leagues[0])),
    engine.save(Team(None, "Ramble", leagues[1])),
    engine.save(Team(None, "Blue", leagues[0])),
    engine.save(Team(None, "Green", leagues[1])),
    ]
players = [
    alice:=engine.save(Athlete(None, "Alice", teams[0])),
    engine.save(Athlete(None, "Bob", teams[0])),
    engine.save(Athlete(None, "Charlie", teams[1])),
    engine.save(Athlete(None, "Dave", teams[2])),
    engine.save(Athlete(None, "Beth", teams[3])),
    engine.save(Athlete(None, "Frank", teams[2])),
]
engine.connection.commit()

In [ ]:
engine.get(Athlete, alice.id).team.league.leaguename

## Alternate Models
Create a model that queries a subset or alternate form of the data, for example pulling in a foreign key as an int id instead of the full `Model` instance.

The name of the table comes before a '_'

In [ ]:
class Team_NameOnly(NamedTuple):
    id: int | None
    teamname: str

engine.get(Team_NameOnly, alice.team.id)

In [ ]:
class Athlete_TeamAsIntId(NamedTuple):
    id: int | None
    name: str
    team: int

engine.get(Athlete_TeamAsIntId, alice.id)

# Querying

`select` from the `query` module is a powerful query builder tool that allows you to incrementally build a query.

The most simple case selects all rows from a table.

`select` returns a tuple of `(Model, query)`. This is also the interface for `Engine.query`

In [ ]:
from micro_namedtuple_sqlite_persister.query import select

M, q = select(Athlete)

for player in engine.query(M, q).fetchall():
    print(player)


You can inline the `select(Model)` and splat the `(M,q)` tuple right into the query, since you rarely need to access the query string directly.

In [ ]:
for player in engine.query(*select(Athlete)).fetchall():
    print(player)

The select query can easily be modified to add `WHERE` clauses.

In [ ]:
@select(Athlete)
def athletes_named_beth():
    return f"WHERE {Athlete.name} = 'Beth'"

for player in engine.query(*athletes_named_beth()).fetchall():
    print(player)

Joins happen implicity when needed. The are disambiguated by field name.

In [ ]:
# a single join
@select(Athlete)
def athletes_on_red_team():
    return f"WHERE {Athlete.team.teamname} = 'Red'"

# two joins required
@select(Athlete)
def athletes_in_big_leagues():
    return f"WHERE {Athlete.team.league.leaguename} = 'Big'"


for player in engine.query(*athletes_on_red_team()).fetchall():
    print(player)

for player in engine.query(*athletes_in_big_leagues()).fetchall():
    print(player)

Currently you would need to fallback to raw queries if you need backref in predicate

In [ ]:
# e.g. you cannot do this:

@select(League)
def leagues_with_big_teams():
    return f"WHERE {Team.teamname} = 'Big'"

# This won't work because the path based join resolution requires all columns specs to start from the root model, e.g. League
# instead, you can do this:


sql = 'SELECT * FROM League JOIN Team ON Team.league = League.id WHERE Team.teamname = "Big"'
result = engine.query(League, sql).fetchone()


## Query Parameters
The decorated function can also take parameters which will be return as a third element in the tuple.

In [ ]:
@select(Athlete)
def athletes_in_league(league: str):
    return f"WHERE {Athlete.team.league.leaguename} = {league}"

M, q, p = athletes_in_league('Big')

print("Our query:")
print(q)
print(p)
print()

for player in engine.query(M, q, p).fetchall():
    print(player)

# or the same, but more concisely
for player in engine.query(*athletes_in_league('Small')).fetchall():
    print(player)

## SQLite3 Cursor
Notice that query returns a real `sqlite3.Cursor`, you can use it to `fetchall`, `fetchone`, `fetchmany`, etc.

The only thing we do is set the `Cursor.row_factory` to return Model instances, and stub the static typehints in for them as well.

In [ ]:
engine.query(*select(Athlete)).fetchone()

## Arbitrary Queries
It's possible to use models that are not tables. Just provide bespoke SQL queries to the `query` parameter, along with the Model you want returned.

In [ ]:
class AverageScoreResults(NamedTuple):
    avg_score: float
    scorecount: int

sql = 'SELECT avg(score), count(*) FROM MyModel'

result = engine.query(AverageScoreResults, sql).fetchone()
assert result is not None
print(f'The table has {result.scorecount} rows, with and average of {result.avg_score:0.2f}')

# Persisting native python collections
You can persist any list or dict that recusively serializes to valid JSON using the `json.dumps'and `json.loads` methods. Enums will be supported in the future.

In [ ]:
class JsonExample(NamedTuple):
    id: int | None
    names: dict

engine.ensure_table_created(JsonExample)
names = {"Alice": 1, "Bob": 2, "Charlie": 3}
row = engine.insert(JsonExample(None, names))

engine.get(JsonExample, row.id)

## SQLite3 supports JSON extensions

In [ ]:
class Character(NamedTuple):
    id: int | None
    name: str
    stats: dict

engine.ensure_table_created(Character, force_recreate=True)

engine.insert(Character(None, 'Harbel', {'spell': 'Fireball', 'level': 3}))
engine.insert(Character(None, 'Quenswen', {'spell': 'Waterspout', 'level': 27}))
engine.insert(Character(None, 'Ruthbag', {'spell': 'Fireball', 'level': 12}))

@select(Character)
def get_fireball_characters():
    f"WHERE {Character.stats} ->> '$.spell' = 'Fireball'"

for c in engine.query(*get_fireball_characters()).fetchall():
    print(f"{c.name} has a fireball at level {c.stats['level']}")

# Persisting Custom Types: Adapt/Convert

In [ ]:
import pandas as pd
import pickle

from micro_namedtuple_sqlite_persister.adaptconvert import register_adapt_convert

def adapt_df(obj: pd.DataFrame) -> bytes:
    return pickle.dumps(obj)


def convert_df(data: bytes) -> pd.DataFrame:
    return pickle.loads(data)


register_adapt_convert(pd.DataFrame, adapt_df, convert_df, overwrite=True)


class MyModel2(NamedTuple):
    id: int | None
    name: str
    df: pd.DataFrame

engine.ensure_table_created(MyModel2)

df = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6]})
row = engine.insert(MyModel2(None, "foo", df))

engine.get(MyModel2, row.id).df


# Performance scenarios
Every call to insert real full trip to the db. The data is ready to be queried immediately, in SQLAlchemy parlance, 'flushed'. Committig ends the implicit transaction and ensures that the data is persisted to disk. Data is then avialable to other connections e.g. other worker processes

Because the db and app share a process, the performance is good enough that you can basically ignore the N+1 problem. This also simplifies implementation of this library, no need to track session etc. It also simplifies your app as data is syncronized immediately with the database, thus eliminates the need for a stateful cache, a source off many bugs and complexity.

In [ ]:
engine.connection.set_trace_callback(None) # disable echo SQL

## Insert Many (17,000 rows)

In [ ]:
for i in range(17000):
    engine.insert(MyModel(None, "foo", dt.datetime.now(), random()*100))

engine.connection.commit()

## Update many (17,000 rows)

In [ ]:
for id in range(1, 17000):
    engine.update(MyModel(id, "drew", dt.datetime.now(), random()*100))

engine.connection.commit()

## Query many

In [ ]:
def print_30_per_line(ss: Iterable[str]):
    for i,s in enumerate(ss, 1):
        print(s, end=" ")
        if i % 30 == 0:
            print()
    print()

@select(MyModel)
def high_scores():
    return f"WHERE {MyModel.score} > 95.7"

rows = engine.query(*high_scores()).fetchall()
print_30_per_line(f"{r.score:5.1f}" for r in rows)

## Giant Recursive BOM

In [ ]:
class BOM(NamedTuple):
    id: int | None
    name: str
    value: float
    child_a: BOM | None
    child_b: BOM | None

engine.ensure_table_created(BOM, force_recreate=True)

from random import random, choice
node_count = 0
def generate_node_name_node(depth: int) -> str:
    alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    return f"{choice(alphabet)}{choice(alphabet)}{choice(alphabet)}{depth:05d}_{node_count}"


# create a giant BOM, of 15 levels deep
def create_bom(depth: int) -> BOM:
    global node_count
    node_count += 1

    if depth == 1:
        child_a = None
        child_b = None
    else:
        child_a = create_bom(depth-1)
        child_b = create_bom(depth-1)

    return BOM(None, generate_node_name_node(depth), random()*1000 - 500, child_a, child_b)

root = create_bom(13)
print(f"Created a BOM with {node_count} nodes")

In [ ]:
inserted_root = engine.insert(root)
engine.connection.commit()

print(f"Inserted BOM with id: {inserted_root.id}")

In [ ]:
recovered_root = engine.get(BOM, inserted_root.id)

def count_nodes(node: BOM | None) -> int:
    if node is None:
        return 0
    return 1 + count_nodes(node.child_a) + count_nodes(node.child_b)

print(f"Recovered BOM with {count_nodes(recovered_root)} nodes")

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def add_nodes_edges(G: nx.Graph, node: BOM | None):
    if node is None:
        return

    G.add_node(node.id, label=node.name)
    if node.child_a is not None:
        G.add_edge(node.id, node.child_a.id)
        add_nodes_edges(G, node.child_a)

    if node.child_b is not None:
        G.add_edge(node.id, node.child_b.id)
        add_nodes_edges(G, node.child_b)

G = nx.Graph()
add_nodes_edges(G, recovered_root)

pos = nx.nx_agraph.graphviz_layout(G, prog="twopi", args="")
plt.figure(figsize=(10, 10))
def alpha(x, a=-.8, b=6.5):
    import math
    return 1 / (1 + math.exp(-a * (math.log(x) - b)))
nx.draw(G, pos, node_size=10, alpha=alpha(node_count), node_color="blue", with_labels=node_count<1200, labels=nx.get_node_attributes(G, "label"))
plt.axis("equal")
plt.show()

In [ ]:
# use an alt model to be able to query the BOM table without recursively pulling in children for EVERY row
class BOM_ChildrenAsId(NamedTuple):
    id: int | None
    name: str
    value: float
    child_a: int | None
    child_b: int | None

engine.query(*select(BOM_ChildrenAsId)).fetchall()

# Error Scenarios

In [ ]:
# inserting a row with an id that already exists will raise an error
engine.insert(MyModel(row.id, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Trying to update a row without specifying an id will raise an error
engine.update(MyModel(None, "bar", dt.datetime.now(), 3.14))

In [ ]:
# Raises an error if the id does not exist
engine.update(MyModel(878787879879, "bar", dt.datetime.now(), 3.14))

In [ ]:
# If schema already exists, but is not correct

class MyModelExists(NamedTuple): # type: ignore this is part of the error
    id: int | None
    name: str
engine.ensure_table_created(MyModelExists)

class MyModelExists(NamedTuple):
    id: int | None
    name: str | None
engine.ensure_table_created(MyModelExists)

In [ ]:
# you have to have id: `int | None` as the first field

class MyModelMissingId(NamedTuple):
    name: str

engine.ensure_table_created(MyModelMissingId)

# Meta
Right now this is just debugging internals

In [ ]:
from micro_namedtuple_sqlite_persister.model import _meta
for k,v in _meta.items():
    print(f"{k}: {v}")